In [1]:
import sys
import cv2
from PyQt5.QtWidgets import (QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, 
                             QWidget, QFileDialog, QLineEdit, QSplitter, QMessageBox)
from PyQt5.QtGui import QPixmap, QImage, QFont
from PyQt5.QtCore import Qt, QTimer
from ultralytics import YOLO

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("YOLOv8 Object Counter Pro")
        self.setGeometry(100, 100, 1000, 600)

        # تم جدید: روشن، حرفه‌ای و زیبا
        self.setStyleSheet("""
            QMainWindow { 
                background-color: #E8F0F5; 
                font-family: Segoe UI, Arial;
            }
            QLabel { 
                color: #2D3748; 
                font-size: 14px;
            }
            QLineEdit { 
                background-color: #FFFFFF; 
                color: #2D3748; 
                border: 2px solid #A0AEC0; 
                border-radius: 8px; 
                padding: 8px; 
                font-size: 14px;
                font-weight: bold;
            }
            QPushButton {
                background-color: #4C51BF;
                color: white;
                border: none;
                padding: 12px;
                font-size: 15px;
                font-weight: bold;
                border-radius: 10px;
                min-width: 180px;
            }
            QPushButton:hover {
                background-color: #5A67D8;
            }
            QPushButton:pressed {
                background-color: #434190;
            }
            QSplitter::handle {
                background-color: #CBD5E0;
                width: 3px;
            }
        """)

        # تقسیم صفحه با QSplitter
        splitter = QSplitter(Qt.Horizontal)
        self.setCentralWidget(splitter)

        # --- سمت چپ: نمایش تصویر + باکس شمارش ---
        left_widget = QWidget()
        left_layout = QVBoxLayout(left_widget)

        self.display_label = QLabel("Image or video will appear here")
        self.display_label.setAlignment(Qt.AlignCenter)
        self.display_label.setStyleSheet("""
            background-color: #FFFFFF; 
            border: 2px solid #CBD5E0; 
            border-radius: 12px;
            font-size: 16px;
            color: #718096;
        """)
        self.display_label.setMinimumSize(600, 400)
        left_layout.addWidget(self.display_label, stretch=1)

        # باکس شمارش
        self.count_box = QLineEdit()
        self.count_box.setReadOnly(True)
        self.count_box.setPlaceholderText("Object count: 0")
        self.count_box.setFont(QFont("Segoe UI", 12, QFont.Bold))
        self.count_box.setStyleSheet("background-color: #EDF2F7; border: none; padding: 10px; border-radius: 8px;")
        left_layout.addWidget(self.count_box)

        splitter.addWidget(left_widget)

        # --- سمت راست: دکمه‌ها ---
        right_widget = QWidget()
        right_layout = QVBoxLayout(right_widget)
        right_layout.setSpacing(15)
        right_layout.setContentsMargins(20, 40, 20, 20)

        # دکمه‌های انگلیسی
        load_image_btn = QPushButton("Load Image")
        load_image_btn.clicked.connect(self.load_image)
        right_layout.addWidget(load_image_btn)

        load_video_btn = QPushButton("Load Video")
        load_video_btn.clicked.connect(self.load_video)
        right_layout.addWidget(load_video_btn)

        self.camera_btn = QPushButton("Turn On Camera")
        self.camera_btn.clicked.connect(self.toggle_camera)
        right_layout.addWidget(self.camera_btn)

        self.detect_btn = QPushButton("Start Detection")
        self.detect_btn.clicked.connect(self.toggle_detection)
        right_layout.addWidget(self.detect_btn)

        right_layout.addStretch()
        splitter.addWidget(right_widget)

        # متغیرها
        self.cap = None
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        try:
            self.model = YOLO("yolov8-drone.pt")
        except Exception as e:
            QMessageBox.warning(self, "Error", f"Model loading failed: {e}\nCheck your model file.")
            self.model = None

        self.detecting = False
        self.is_camera = False
        self.is_video = False
        self.current_frame = None
        self.video_path = None

    def load_image(self):
        self.stop_stream()
        file_name, _ = QFileDialog.getOpenFileName(self, "Load Image", "", "Image Files (*.png *.jpg *.jpeg *.bmp)")
        if file_name:
            self.current_frame = cv2.imread(file_name)
            self.is_camera = False
            self.is_video = False
            self.display_frame(self.current_frame)

    def load_video(self):
        self.stop_stream()
        file_name, _ = QFileDialog.getOpenFileName(self, "Load Video", "", "Video Files (*.mp4 *.avi *.mov *.mkv)")
        if file_name:
            self.video_path = file_name
            self.cap = cv2.VideoCapture(file_name)
            if not self.cap.isOpened():
                self.count_box.setText("Failed to open video")
                return
            self.is_video = True
            self.is_camera = False
            self.timer.start(30)

    def toggle_camera(self):
        if not self.is_camera:
            self.stop_stream()
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                self.count_box.setText("Camera not available")
                return
            self.timer.start(30)
            self.camera_btn.setText("Turn Off Camera")
            self.is_camera = True
            self.is_video = False
        else:
            self.stop_stream()
            self.camera_btn.setText("Turn On Camera")
            self.is_camera = False

    def toggle_detection(self):
        if self.model is None:
            QMessageBox.warning(self, "Error", "Model not loaded.")
            return
        self.detecting = not self.detecting
        if self.detecting:
            self.detect_btn.setText("Stop Detection")
            if not self.is_camera and not self.is_video and self.current_frame is not None:
                self.process_and_display(self.current_frame)
        else:
            self.detect_btn.setText("Start Detection")
            self.count_box.setText("Object count: 0")
            if not self.is_camera and not self.is_video and self.current_frame is not None:
                self.display_frame(self.current_frame)

    def update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            self.process_and_display(frame)
        else:
            if self.is_video:
                self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    def process_and_display(self, frame):
        if self.model is not None and self.detecting:
            results = self.model(frame)
            annotated_frame = results[0].plot()
            count = len(results[0].boxes) if results[0].boxes is not None else 0
            self.count_box.setText(f"Object count: {count}")
            self.display_frame(annotated_frame)
        else:
            self.display_frame(frame)
            self.count_box.setText("Object count: 0")

    def display_frame(self, frame):
        if frame is not None:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            height, width, channel = frame_rgb.shape
            bytes_per_line = channel * width
            q_img = QImage(frame_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_img).scaled(
                self.display_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            )
            self.display_label.setPixmap(pixmap)

    def stop_stream(self):
        self.timer.stop()
        if self.cap:
            self.cap.release()
            self.cap = None
        self.display_label.clear()
        self.display_label.setText("Image or video will appear here")
        self.count_box.setText("Object count: 0")


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())


0: 448x640 1 drone, 720.6ms
Speed: 108.2ms preprocess, 720.6ms inference, 71.2ms postprocess per image at shape (1, 3, 448, 640)


SystemExit: 0

F:\user\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
